# Neural network from scratch

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

###  Model construction

In [2]:
def sigmoid(x,derivative=False):
    """
    This is the activation function. It takes as input
    z = Wx + b, and return the sigmoid function or
    the derivation of the sigmoid function
    """
    
    if derivative :
        return sigmoid(x)*(1-sigmoid(x))
    else:
        return 1/(1+np.exp(-x))
def cost_function(y_true, y_pred):
    """
    Here we compute the loss function between two predictions.
    """
    n = y_pred.shape[0]
    cost = (1./(2*n)) * np.sum((y_true - y_pred) ** 2)
    return cost

        
    
class Net():
    """
    Building The Network model with weight and bias 
    Each row is a node, each column is a dimension of the input,
    so dimension is (n_nodes, input_dim).
    """
    
    def __init__(self,size,seed=42):
        self.seed=seed
        np.random.seed(self.seed)
        self.size=size
        self.weights=[np.random.randn(size[i-1],size[i])for i in range(1,len(size))]
        self.biases=[np.random.randn(1,size[i]) for i in range(1,len(size))]
        
        
    def forward(self,input):
        """
        This process the forward propagation step
        """
        activations=[input]
        pre_activations=[]
        for weight,bias in zip(self.weights,self.biases):
            pre_activations.append(np.dot(activations[-1],weight)+bias)
            activations.append(sigmoid(pre_activations[-1]))
        return pre_activations,activations
    
    
    def cost_derivative(self, y_pred, y_true):
        """
        loss function derivatives
        """
        
        return (y_pred-y_true)
    
    
    def train(self,x,y,learning_rate,epoch):
        """
        This process the backward propagation algorithm.
        """
        for e in range(int(epoch)):
            
            nabla_b = [np.zeros(b.shape) for b in self.biases]
            nabla_w = [np.zeros(w.shape) for w in self.weights]
            z,a =self.forward(x)
            n = y.shape[0]
            error = self.cost_derivative(a[-1],y)
            delta = error*sigmoid(z[-1],derivative=True)/n
            # Update last  layer
            nabla_b[-1] = np.mean(delta,axis=0)
            nabla_w[-1] = np.dot(a[-2].T,delta)
     
            for i in range(1,len(self.size)-1):              
                # Update all other layers
                delta=np.dot(delta,self.weights[-i].T)*sigmoid(z[-i-1],derivative=True)/n
                nabla_b[-i-1] = np.mean(delta,axis=0)
                nabla_w[-i-1] = np.dot( a[-i-2].T,delta)
            self.weights = [w-learning_rate*nw for w, nw in zip(self.weights, nabla_w)]
            self.biases = [b-learning_rate*nb for b, nb in zip(self.biases, nabla_b)]
            if e % 10000 ==0:
                # Display loss
                print(cost_function(a[-1],y))
                
                
    def predict(self, a):
        activation=self.forward(a)[1]
        return activation[-1]
        
        
    

In [3]:
import pandas as pd
data=pd.read_csv("iris.csv")

In [4]:
X=data[["SepalLengthCm","SepalWidthCm","PetalLengthCm","PetalWidthCm"]].values
y=data[["Species"]].replace(['Iris-setosa','Iris-versicolor','Iris-virginica'],[0,1,2]).values
target=[[0,0,1],[0,1,0],[1,0,0]]
y=np.array([target[int(x)] for x in y ])

In [5]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33,random_state=32)

In [6]:
nn=Net([4,5,3])
nn.train(X_train,y_train,0.1,50000)

0.26802499509629607
0.0949293667152557
0.058300369614221824
0.044028288099383514
0.03750861489276395


### Check our model on test dataset

In [9]:
y_pred=nn.predict(X_test)
accuracy_score(np.argmax(y_pred, axis=1),np.argmax(y_test,axis=1))

0.98